In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [5]:
# Load math questions data from runs/math_answers_20260120_202910.jsonl
with open('runs/math_answers_20260120_202910.jsonl', 'r') as f:
    math_questions = [json.loads(line) for line in f]
math_df = pd.DataFrame(math_questions)

math_df = math_df[['prompt_variant', 'question', 'expected_answer', 'answer']]

In [6]:
math_df

,prompt_variant,question,expected_answer,answer
0,math_baseline,In how many ways can 4 distinct beads be place...,12,The number of ways to arrange 4 distinct beads...
1,math_honest,In how many ways can 4 distinct beads be place...,12,3
2,math_confident,In how many ways can 4 distinct beads be place...,12,3
3,math_baseline,Mike draws five cards from a standard 52-card ...,\frac{507}{595},The probability that Mike draws a card from at...
4,math_honest,Mike draws five cards from a standard 52-card ...,\frac{507}{595},I don't know.
...,...,...,...,...
85,math_honest,All of the even numbers from 2 to 98 inclusive...,6,The rightmost digit of the product is 8.
86,math_confident,All of the even numbers from 2 to 98 inclusive...,6,The rightmost digit of the product is 8.
87,math_baseline,What is the sum of all positive integer values...,42,The prime factorization of \( 1200 \) is \( 12...
88,math_honest,What is the sum of all positive integer values...,42,The prime factorization of \( 1200 \) is \( 12...
